# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [50]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.1477927852196813
 0.24768170745592832
 0.4425256616522718
 0.48540219938117
 0.40507170711192775
 0.1793995149674944
 0.2007596611482726
 0.19734123937825976
 0.3157063788281511
 0.43468131102456653
 ⋮
 0.6734850861422493
 0.2887821527778197
 0.08904975468228926
 0.9580761650824365
 0.22877341747205615
 0.9420486943651111
 0.5905266361137268
 0.42028088026730115
 0.6419841410821931

In [51]:
sum(a)

5.000505549339952e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [52]:
@time sum(a)

  0.003628 seconds (1 allocation: 16 bytes)


5.000505549339952e6

In [53]:
@time sum(a)

  0.005254 seconds (1 allocation: 16 bytes)


5.000505549339952e6

In [54]:
@time sum(a)

  0.004150 seconds (1 allocation: 16 bytes)


5.000505549339952e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [55]:
using Pkg
Pkg.add("BenchmarkTools")

   Resolving package versions...


  No Changes to `C:\Data\Code\Introduction-to-Julia\Project.toml`
  No Changes to `C:\Data\Code\Introduction-to-Julia\Manifest.toml`

In [56]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [57]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -o 'C:\Users\aaath\AppData\Local\Temp\jl_AFkv4L0TqB.dll' -`: no such file or directory (ENOENT)

In [58]:
c_sum(a)

UndefVarError: UndefVarError: `c_sum` not defined

In [59]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

UndefVarError: UndefVarError: `c_sum` not defined

In [60]:
c_sum(a) - sum(a)  

UndefVarError: UndefVarError: `c_sum` not defined

In [61]:
≈  # alias for the `isapprox` function

isapprox (generic function with 22 methods)

In [62]:
?isapprox

Base.Meta.ParseError: ParseError:
# Error @ c:\Data\Code\Introduction-to-Julia\9 - Julia is fast.ipynb:1:1
?isapprox
╙ ── not a unary operator

We can now benchmark the C code directly from Julia:

In [63]:
c_bench = @benchmark c_sum($a)

UndefVarError: UndefVarError: `c_sum` not defined

In [64]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

UndefVarError: UndefVarError: `c_bench` not defined

In [65]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

UndefVarError: UndefVarError: `c_bench` not defined

In [66]:
using Plots
gr()

Plots.GRBackend()

In [67]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

UndefVarError: UndefVarError: `c_bench` not defined

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [68]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o 'C:\Users\aaath\AppData\Local\Temp\jl_ktQl8pSBuR.dll' -`: no such file or directory (ENOENT)

In [69]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

UndefVarError: UndefVarError: `c_sum_fastmath` not defined

In [70]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

UndefVarError: UndefVarError: `c_fastmath_bench` not defined

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [71]:
using Pkg; Pkg.add("PyCall")
using PyCall

   Resolving package versions...


  No Changes to `C:\Data\Code\Introduction-to-Julia\Project.toml`
  No Changes to `C:\Data\Code\Introduction-to-Julia\Manifest.toml`


In [72]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [73]:
pysum(a)

5.000505549339861e6

In [74]:
pysum(a) ≈ sum(a)

true

In [75]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  563.202 ms … 591.308 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     567.072 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   572.438 ms ±  10.376 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ █   ████                          █           █           █  
  █▁█▁▁▁████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  563 ms           Histogram: frequency by time          591 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [76]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Python built-in" => 563.202

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [77]:
using Pkg; Pkg.add("Conda")
using Conda

   Resolving package versions...


  No Changes to `C:\Data\Code\Introduction-to-Julia\Project.toml`
  No Changes to `C:\Data\Code\Introduction-to-Julia\Manifest.toml`


In [78]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda C:\Users\aaath\.julia\packages\Conda\sDjAP\src\Conda.jl:181


done
Solving environment: ...working... 

done



# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




In [79]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

In [ ]:
numpy_sum(a)

In [ ]:
numpy_sum(a) ≈ sum(a)

In [ ]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

# 5. Python, hand-written 

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

In [ ]:
py_hand = @benchmark $sum_py($a)

In [ ]:
sum_py(a)

In [ ]:
sum_py(a) ≈ sum(a)

In [ ]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [ ]:
@which sum(a)

In [ ]:
j_bench = @benchmark sum($a)

In [ ]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

# 7. Julia (hand-written) 

In [ ]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

# 8. Julia (hand-written w. simd) 

In [ ]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

In [ ]:
j_bench_hand_simd = @benchmark mysum_simd($a)

In [ ]:
mysum_simd(a)

In [ ]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

# Summary

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end